# The 2018 Midterms

In [1]:
import pandas as pd
import numpy as np

## Context

In [2]:
elec2018 = pd.read_csv('../Datasets/2018-elections/election-context-2018.csv').set_index(['state', 'county'])
elec2018.head()

fips  trump16  clinton16  otherpres16  romney12  obama12  \
state   county                                                              
Alabama Autauga  1001    18172       5936          865     17379     6363   
        Baldwin  1003    72883      18458         3874     66016    18424   
        Barbour  1005     5454       4871          144      5550     5912   
        Bibb     1007     6738       1874          207      6132     2202   
        Blount   1009    22859       2156          573     20757     2970   

                 otherpres12  demsen16  repsen16  othersen16      ...        \
state   county                                                    ...         
Alabama Autauga          190    6331.0   18220.0        62.0      ...         
        Baldwin          898   19145.0   74021.0       248.0      ...         
        Barbour           47    4777.0    5436.0        16.0      ...         
        Bibb              86    2082.0    6612.0        17.0      ...         
        Blount           279    2980.0   22169.0        48.0      ...         

                 age29andunder_pct  age65andolder_pct  median_hh_inc  \
state   county                                                         
Alabama Autauga          40.037058          13.978456        53099.0   
        Baldwin          35.474412          18.714851        51365.0   
        Barbour          37.664387          16.528895        33956.0   
        Bibb             37.329435          14.885699        39776.0   
        Blount           37.240053          17.192916        46212.0   

                 clf_unemploy_pct  lesshs_pct  lesscollege_pct  \
state   county                                                   
Alabama Autauga          5.591657   12.417046        75.407229   
        Baldwin          6.286843    9.972418        70.452889   
        Barbour         12.824738   26.235928        87.132213   
        Bibb             7.146827   19.301587        88.000000   
        Blount           5.953833   19.968585        86.950243   

                 lesshs_whites_pct  lesscollege_whites_pct  rural_pct  \
state   county                                                          
Alabama Autauga          10.002112               74.065601  42.002162   
        Baldwin           7.842227               68.405607  42.279099   
        Barbour          19.579752               81.364746  67.789635   
        Bibb             15.020490               87.471774  68.352607   
        Blount           16.643368               86.163610  89.951502   

                 ruralurban_cc  
state   county                  
Alabama Autauga            2.0  
        Baldwin            3.0  
        Barbour            6.0  
        Bibb               1.0  
        Blount             1.0  

[5 rows x 37 columns]

### What Statistics Do We Have?
What kind of contextual information do we have here? Let's list the column labels and their respective indexes. (Note that I've cheated a bit and limited the list to make it managabe; everything after the 19th index is the same `pop_stats` slice.)

In [3]:
list(enumerate(elec2018.columns))[:20]

[(0, 'fips'),
 (1, 'trump16'),
 (2, 'clinton16'),
 (3, 'otherpres16'),
 (4, 'romney12'),
 (5, 'obama12'),
 (6, 'otherpres12'),
 (7, 'demsen16'),
 (8, 'repsen16'),
 (9, 'othersen16'),
 (10, 'demhouse16'),
 (11, 'rephouse16'),
 (12, 'otherhouse16'),
 (13, 'demgov16'),
 (14, 'repgov16'),
 (15, 'othergov16'),
 (16, 'repgov14'),
 (17, 'demgov14'),
 (18, 'othergov14'),
 (19, 'total_population')]

#### Some Useful Index Slices

In [4]:
# All presidential slices
pres = elec2018.columns[1:7]
pres16 = pres[:3]
pres12 = pres[3:]

# All congressional slices
congress = elec2018.columns[7:13]
senate = congress[:3]
house = congress[3:]

# All gubernatorial slices
gov = elec2018.columns[13:19]
gov16 = gov[:3]
gov14 = gov[3:]

# Partisan slices
dem = [ind for ind in elec2018.columns if 'dem' in ind] + ['clinton16', 'obama12']
rep = [ind for ind in elec2018.columns if 'rep' in ind] + ['trump16', 'romney12']

# Population slices
pop_stats = elec2018.columns[19:]

#### Romney-Clinton Counties

In [5]:
rom_clin = elec2018[(elec2018['clinton16'] > elec2018['trump16'])
                    & (elec2018['romney12'] > elec2018['obama12'])]
rom_clin.loc[:, pres].head()

trump16  clinton16  otherpres16  romney12  obama12  \
state      county                                                         
California Nevada      23365      26053         5517     24986    24663   
           Orange     507148     609961        80412    582332   512440   
Georgia    Cobb       152912     160121        17786    171722   133124   
           Gwinnett   146989     166153        15189    159855   132509   
           Henry       45724      50057         2985     46774    43761   

                     otherpres12  
state      county                 
California Nevada           2027  
           Orange          27892  
Georgia    Cobb             5270  
           Gwinnett         4364  
           Henry             843

#### Obama-Trump Counties

In [6]:
obama_trump = elec2018[(elec2018['clinton16'] < elec2018['trump16'])
                    & (elec2018['romney12'] < elec2018['obama12'])]
obama_trump.loc[:, pres].head()

trump16  clinton16  otherpres16  romney12  obama12  \
state    county                                                         
Alabama  Barbour      5454       4871          144      5550     5912   
         Conecuh      3420       3080           85      3439     3555   
Arkansas Woodruff     1347       1118          106      1227     1340   
Colorado Chaffee      5391       4888          971      5070     5086   
         Conejos      1914       1771          337      1835     2213   

                   otherpres12  
state    county                 
Alabama  Barbour            47  
         Conecuh            28  
Arkansas Woodruff          118  
Colorado Chaffee           306  
         Conejos            53

In [8]:
flips = rom_clin.append(obama_trump, sort=True)
flips.loc[:, pres]

TypeError: append() got an unexpected keyword argument 'sort'

#### Largest Flips in the Nations
Let's find which counties showed the change relative to the size of their electorate.

In [9]:
rom_clin16 = rom_clin.loc[:, pres16].sum(1)
rom_clin12 = rom_clin.loc[:, pres12].sum(1)
rom_clin12.head()

state       county  
California  Nevada        51676
            Orange      1122664
Georgia     Cobb         310116
            Gwinnett     296728
            Henry         91378
dtype: int64

#### The Electorates
First we'll need to get a sum of the entire electorate in these counties. To do this we will get the sum of all three presidential columns for each election. Recall that this includes third party candidates.

In [10]:
obama_trump16 = obama_trump.loc[:, pres16].sum(1)
obama_trump12 = obama_trump.loc[:, pres12].sum(1)
obama_trump12.head()

state     county  
Alabama   Barbour     11509
          Conecuh      7022
Arkansas  Woodruff     2685
Colorado  Chaffee     10462
          Conejos      4101
dtype: int64

#### Relative Differences
We get the relative differences by taking the percentage of the votes of each electorate the candidates earned and subtracting those two numbers. Because we have third party candidates, there is a difference between the voters who went _away_ from one party and the candidates who went _to_ another. This is illustrated below. If we had only summated the votes from the two major parties, the two lists below would be identical.

In [11]:
clinton_loss = abs((obama_trump['clinton16'] / obama_trump16) - (obama_trump['obama12'] / obama_trump12))
clinton_loss.sort_values(ascending=False)[:10]

state         county   
Kentucky      Elliott      0.235178
Iowa          Howard       0.228066
Illinois      Henderson    0.226061
Iowa          Jackson      0.204580
              Worth        0.203871
Vermont       Essex        0.201660
Minnesota     Swift        0.199805
North Dakota  Ransom       0.198579
Iowa          Chickasaw    0.195982
North Dakota  Sargent      0.188104
dtype: float64

In [12]:
trump_gain = abs((obama_trump['trump16'] / obama_trump16) - (obama_trump['romney12'] / obama_trump12))
trump_gain.sort_values(ascending=False)[:10]

state      county    
Kentucky   Elliott       0.231163
Iowa       Howard        0.186276
Illinois   Henderson     0.180669
New York   Oswego        0.179756
Iowa       Worth         0.158202
           Jackson       0.157058
New York   Franklin      0.156666
Minnesota  Swift         0.154164
New York   Montgomery    0.150676
           Cayuga        0.150416
dtype: float64

Notice that there is enough difference for counties to move up and down in the sorted lists.

In [13]:
(trump_gain - clinton_loss).sort_values(ascending=False).head()

state     county    
New York  Suffolk       0.055395
          Montgomery    0.051070
          Niagara       0.048477
          Richmond      0.046134
          Rensselaer    0.041925
dtype: float64

In [14]:
(clinton_loss - trump_gain).sort_values(ascending=False).head()

state         county  
Vermont       Essex       0.102799
North Dakota  Ransom      0.101732
              Benson      0.094309
              Sargent     0.081254
New Mexico    Valencia    0.077951
dtype: float64